# **MATH&ML-15. Рекомендательные системы. Часть II**

# 1. Введение

✍ В предыдущем модуле мы начали знакомиться с рекомендательными системами — кратко разобрали существующие подходы к их построению и научились оценивать качество РС. Более подробно мы пока успели изучить только popularity-based подход. В этом модуле мы рассмотрим остальные алгоритмы и научимся реализовывать их на практике.

Для начала давайте кратко повторим материал, изученный в прошлом модуле ↓

Отлично! Теперь вы точно готовы к дальнейшему изучению рекомендательных систем.

Основные цели, которые стоят перед нами в этом модуле:

1. Разобрать принципы работы моделей рекомендательных систем:
    * content-based-модели,
    * коллаборативной фильтрации,
    * гибридной модели.
2. Познакомиться с основами применения глубокого обучения для построения рекомендательных систем.
3. Отработать изученные алгоритмы на решении практических задач.

В [юните «Практика»](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DST-3.0+28FEB2021/jump_to_id/9af88f62b92b44559b442bfcc9f4d23d) мы достроим начатую в предыдущем модуле модель рекомендательной системы для статей, а в промежуточных юнитах рассмотрим применение методов построения рекомендательных систем на других датасетах из различных областей.

# 2. Content-based model

✍ В этом юните мы рассмотрим систему рекомендаций на основе контента, или, как её чаще называют, **content-based model**.

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/392c8cc2-df07-48b3-88f4-088149cb53fe.png)

> Подход **content-based** предполагает, что пользователю рекомендуются товары или контент на основе его предпочтений и вкусов.

Профиль интересов пользователя формируется исходя из его оценок, а также неявной обратной связи: лайков, количества просмотров и так далее. В качестве рекомендаций пользователю предлагаются похожие элементы.

Сходство или близость элементов измеряется на основе сходства содержания этих элементов. Говоря «содержание», мы имеем в виду такие сущности, как категория, тег, жанр и т. д., то есть метаданные. Далее в этом юните мы познакомимся с некоторыми алгоритмами определения сходства.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/792c9ad6bbaf1201d6cfecf6d816838f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_1.png)

Вы наверняка уже сталкивались с content-based-рекомендациями в тех или иных сервисах. Например, на сайте Netflix можно использовать фильтрацию по контенту, чтобы создавать рекомендации из аналогичных элементов, которые размещаются в разделе More Like This.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/a8a0464a8baddb3f5b705ff2dc2f4945/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_2.png)

Давайте на примере рассмотрим построение рекомендательной системы на основе контента для конкретного пользователя.
Допустим, пользователь Михаил выставил лайки и дизлайки для фильмов на одном из веб-сервисов:

**ФИЛЬМ**|**РЕЙТИНГ**
-|-
«Миссия невыполнима»|Лайк
«Джеймс Бонд»|Лайк
«Приключения Буратино»|Дизлайк

Предположим, что по правилам сервиса лайк прибавляет 4.5 балла к фильмам с таким жанром, а дизлайк вычитает 6 баллов. Теперь создадим вектор пользователя для Михаила на основе трёх его оценок:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/118b2ca3c07dba4356d56b26cb692aec/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_3.png)

Присваиваем значение 9 боевикам, так как Михаил поставил лайк двум фильмам с жанром «боевик». Михаил не смотрел анимационные фильмы, так что присваиваем 0 анимации, и, поскольку он оставил плохой отзыв фильму в жанре «дети», присваиваем -6 детским фильмам.

Таким образом, вектор пользователя для Михаила — это $(9,0,-6)$ для шкал (Боевик, Анимация, Дети).
Теперь попробуем предсказать отношение Михаила к фильмам, которые он ещё не смотрел, например «Звёздные войны» и «История игрушек».

«Звёздные войны» — это боевик, который не относится к анимации или детским фильмам, поэтому по шкалам (Боевик, Анимация, Дети) у этого фильма будут координаты $(1,0,0)$. У фильма «История игрушек» будут координаты $(0,1,1)$, так как он относится и к жанру детских фильмов, и к анимации.
Теперь нам необходимо найти произведения вектора пользователя и вектора фильма: чем больше будет скалярное произведение, тем более подходящим для Михаила будет фильм.

Скалярное произведение для «Истории игрушек» равно $-6$, а для «Звёздных войн» оно равно $9$. Следовательно, из этих двух вариантов именно «Звёздные войны» будут рекомендованы Михаилу, что вполне логично, ведь ему больше нравятся боевики.

Аналогично можно вычислить скалярные произведения векторов для всех фильмов на сайте и рекомендовать Михаилу десять наиболее подходящих фильмов.

В данном примере мы опирались лишь на схожесть фильмов по жанрам: скалярное произведение будет увеличиваться в том случае, если у фильма ненулевая координата для того жанра, который пользователь уже оценил положительно. Однако в реальности всё намного сложнее: например, мы можем использовать одновременно жанр фильма, теги, описание, актёров и т. д. Для этого случая нам нужен способ измерить сходство между набором параметров.

Рассмотрим два наиболее популярных метода измерения такого сходства:

* **индекс Жаккара**,
* **косинусная близость**.

### ИНДЕКС ЖАККАРА

Индекс Жаккара измеряет сходство между двумя наборами A и B как мощность множества пересечения, делённую на мощность множества объединения каких-то характеристик объекта. Его удобно применять для категориальных признаков.

$$J(A,B) = \frac{\left|A \cap B \right|}{\left|A \cup B \right|} = \frac{\left|A \cap B \right|}{\left|A \right| + \left|B \right| - \left|A \cap B \right|}$$

Например, с помощью индекса Жаккара мы можем оценить, насколько похожи фильмы, основываясь на наборах ключевых слов (тегов) для них:

* фильм А : {фантастика, школа, романтика};
* фильм B : {приключения, фантастика, школа};
* фильм C : {ужасы, триллер, драма}.

Мы можем предположить, что фильм A больше похож на фильм B, чем на фильм C, так как фильмы A и B имеют два общих тега (фантастика, школа), в то время как фильмы A и C не имеют ни одного общего тега.

Если бы мы рассматривали вычисление этого индекса для фильмов А и В, то получили бы $\frac{2}{4}$, так как в пересечении два тега, а в объединении — четыре. Для фильмов А и C мы получили бы $0$, так как пересечение множеств их тегов является пустым. Таким образом, показатель близости для фильмов А и В получился бы больше, и это подтвердило бы наши предположения об их большей схожести.

### КОСИНУСНАЯ БЛИЗОСТЬ

Подход с использованием индекса Жаккара помог нам создать интуитивное представление о том, что означает сходство набора категориальных значений. Подход с **косинусным сходством** немного сложнее и применяется для оценки близости массивов с числами. Он требует, чтобы мы представляли объекты в виде вектора.
Например, мы можем представить те же самые фильмы как набор из трёх вещественных чисел:

* фильм $A = (1.1, 2.3, 5.1)$;
* фильм $B = (1.3, 2.1, 4.9)$;
* фильм $C = (5.1, 6.2, 1.1)$.

Глядя на эти векторы, кажется, что фильмы А и В похожи друг на друга больше, чем, например, фильмы А и С, так как координаты фильмов А и В очень близки и различаются гораздо меньше, чем координаты фильмов А и С.

Чтобы вычислить косинусную близость, нам понадобится следующая формула:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/19ae75390814099a1aaf8a660888b0d6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_5.png)

По сути, мы ищем угол между векторами. Если два вектора совпадут (т. е. будут максимально близкими), то угол между ними будет равен нулю, а значит, косинус будет равен $1$. Если векторы будут направлены в противоположные стороны, косинус будет равен $-1$. Таким образом, мы можем получить для любых двух векторов значение в пределах от $1$ до $-1$ включительно, по которому можно определить, насколько векторы близки друг к другу.
Давайте вычислим косинусную близость, чтобы оценить, насколько наши догадки о близости фильмов соответствуют реальности.

Косинусная близость для фильмов А и B:

$\operatorname{sim}(A, B)=\frac{1.1\cdot 1.3+2.3 \cdot 2.1+5 \cdot 1^* 4.9}{\sqrt{1.1^2+2 .3^2+5.1^2} \sqrt{1.3^2+2.1^2+4.9^2}} \approx 0,999$

### Задание 2.1

Вычислите косинусную близость между векторами А и С. Результат округлите до трёх знаков после точки-разделителя.

In [1]:
A = [1.1, 2.3, 5.1]
C = [5.1, 6.2, 1.1]

sim = (A[0]*C[0] + A[1]*C[1] + A[2]*C[2])/ \
    ((A[0]**2 + A[1]**2 + A[2]**2)**0.5 * (C[0]**2+C[1]**2+C[2]**2)**0.5)
    
round(sim, 3)

0.551

Как нам известно, чем выше значение косинусной близости, тем больше сходство между векторами. Таким образом, наши изначальные предположения совпали с реальностью: векторы А и В действительно схожи между собой сильнее, чем векторы А и С. Значит, если человек поставил высокую оценку фильму А, то мы должны порекомендовать ему фильм В.

Итак, чтобы построить рекомендательную систему на основе контента, необходимо:

1. Для каждого продукта создать характеризующие его признаки.
2. Найти показатель близости между всеми продуктами.
3. Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.

Давайте реализуем подобную рекомендательную систему на практике. Будем работать с [датасетом](https://lms-cdn.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/netflix_titles.zip), содержащим информацию об оценивании фильмов на платформе Netflix.

**Признаки в данных:**

* `show_id` — id фильма,
* `type` — его тип (фильм или сериал),
* `title` — название,
* `director` — режиссер,
* `cast` — актерский состав,
* `country` — страна,
* `date_added` — дата добавления,
* `release_year` — год выхода на экраны,
* `rating` — рейтинг,
* `duration` — продолжительность,
* `listened_in` — жанр(-ы),
* `description` — описание.

В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход **TF-IDF** (Term Frequency-Inverse Document Frequency).

> **Показатель TD-IDF** — это индикатор того, насколько релевантно слово в контексте документа.

Его можно определить следующим образом:

$$\text{TF-IDF(слова) = TF(слова) * IDF (слова)}$$

где:

* $\text{TF слова} = \frac{\text{Количество раз, когда слово встретилось в тексте}}{\text{Количество всех слов в тексте}}$

* $\text{IDF слова} = log \left (\frac{\text{Общее кол-во документов}}{\text{Кол-во документов, в которых встречается слово}}\right )$

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

* Коэффициент будет выше, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
* Коэффициент будет ниже, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

Если вам интересно подробнее изучить алгоритм создания такого представления, рекомендуем прочитать [статью](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3).

Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки `sklearn` — импортируем её:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer


Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого определим нашу модель:

In [3]:
model = TfidfVectorizer(stop_words='english')


Заполним пропуски пустыми строками:



In [4]:
import pandas as pd

df = pd.read_csv('data/netflix_titles.csv')

df['description'] = df['description'].fillna('')

Трансформируем наши описания в матрицу:



In [5]:
feature_matrix = model.fit_transform(df['description'])


### Задание 2.2

Сколько столбцов в получившейся матрице?

In [6]:
feature_matrix

<7787x17905 sparse matrix of type '<class 'numpy.float64'>'
	with 107187 stored elements in Compressed Sparse Row format>

Теперь необходимо вычислить косинусную близость. Можно сделать это так:

In [7]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

**Обратите внимание!** Мы используем здесь `linear_kernel()`, а не [`cosine_similarity()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.

Вернём индексацию и уберём дубликаты из данных:



In [8]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()


Теперь пропишем функцию для создания рекомендаций:



In [9]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

Например, если мы хотим найти рекомендации по фильму "Star Trek", то функция будет выдавать следующий результат:

In [10]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

### Задание 2.3

Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

In [11]:
get_recommendations('Balto').iloc[1]

'Vroomiz'

Итак, мы смогли создать рекомендации на основе контента и разобрались с принципом работы этого алгоритма. Теперь давайте посмотрим на преимущества и недостатки данного подхода.

**ПРЕИМУЩЕСТВА**

* **Для создания рекомендаций не требуются данные от других пользователей.** Как только пользователь выполнил поиск, просмотрел несколько продуктов и/или совершил несколько покупок, система фильтрации на основе контента может начать создавать соответствующие рекомендации. Это делает её идеальной для компаний и сервисов, у которых нет огромного количества пользователей для формирования выборки.
* **Рекомендации получаются очень релевантными для пользователя.** Рекомендации на основе контента могут быть в значительной степени адаптированы к интересам пользователя, включая рекомендации по нишевым товарам, поскольку метод основан на сопоставлении характеристик или атрибутов объекта базы данных с интересами пользователя.
* **Рекомендации прозрачны для пользователя.** Высокорелевантные рекомендации создают ощущение понятности алгоритмов для пользователя, повышая уровень его доверия к предлагаемым рекомендациям.
* **Вы избегаете проблемы «холодного старта».** Хотя фильтрация на основе контента требует первоначального ввода данных от пользователей, чтобы начать давать рекомендации, качество ранних рекомендаций обычно намного выше, чем у других подходов.
* **Системы фильтрации на основе содержания обычно проще в создании.** Основная работа заключается в создании характеристик, на основании которых будет вычисляться близость.

**НЕДОСТАТКИ**

* **Отсутствие новизны и разнообразия.** Вполне возможно, что человеку, который любит боевики, могли бы понравиться и фильмы ужасов. Однако мы не сможем ему их порекомендовать, если он сам не отметит подобные фильмы как понравившиеся.
* **Присвоенные характеристики могут быть неверными.** Рекомендации на основе контента хороши настолько, насколько хороши атрибуты (характеристики), при

Итак, мы разобрались с первым персонализированным методом для построения рекомендательных систем. В следующих юнитах мы детально изучим другие алгоритмы.

# 3. Коллаборативная фильтрация

✍ Следующий подход к построению рекомендательных систем, который мы изучим, — это коллаборативная фильтрация. Она основана на поиске сходства между пользователями или между продуктами. Используя этот метод, мы можем прогнозировать рейтинги на основе оценок похожих пользователей или похожих продуктов.

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/8d75cb1e-148b-4ba2-ba3c-aefac1e8a161.png)

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0641f061288f15ee47c7986bf6c9c0fe/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_1.png)

Однако для начала давайте рассмотрим очень часто встречающуюся в рекомендательных системах концепцию — матрицу предпочтений.

Чтобы её получить, расположим в матрице клиентов по строкам, а продукты — по столбцам. На пересечении строк и столбцов разместим оценки, поставленные клиентами соответствующим продуктам: первый клиент поставил второму товару 3, третий клиент поставил первому товару 2 и так далее.

/|ТОВАР 1|ТОВАР 2|ТОВАР 3|ТОВАР 4|ТОВАР 5
-|-|-|-|-|-
КЛИЕНТ 1||3||5|
КЛИЕНТ 2|1||1|1|
КЛИЕНТ 3|2|||3|2
КЛИЕНТ 4||4|||5
КЛИЕНТ 5|5||2|3|4

На основе этих данных мы можем разделить пользователей на кластеры. Чтобы это сделать, можно взять некоторую меру близости для пользователей по их истории оценок и на основе полученных значений объединить пользователей в кластеры таким образом, чтобы похожие пользователи оказались в одной группе, а сильно отличающиеся — в разных. В таком случае оценку пользователя для продукта можно прогнозировать как среднюю оценку пользователей этого кластера, оценивших этот продукт.

Таким образом, если нам необходимо предсказать, как конкретный пользователь оценил фильм, мы анализируем оценки, поставленные данному фильму пользователями, которые принадлежат к тому же кластеру, что и изучаемый пользователь, и усредняем эти оценки. Так получается предсказание оценки фильма для нашего пользователя.

В целом, такой подход можно применять, однако у него есть ряд существенных недостатков:

* Нечего рекомендовать новым/нетипичным пользователям. Если появляется пользователь, который ни на кого не похож, мы не знаем, к какому кластеру его отнести. На начальных стадиях мы определяем его в случайный кластер, и рекомендации в таком случае будут плохими.
* Не учитывается специфика каждого пользователя. По сути, мы выявляем некоторые паттерны поведения и предпочтений и для каждого паттерна выделяем свои рекомендации. Однако на самом деле даже пользователи из одного кластера немного отличаются друг от друга, поэтому возникают неточности.
* Если оценок нет, то среднее арифметическое невозможно вычислить. Если в кластере никто не оценивал объект, сделать предсказание не получится, так как для предсказания нужно вычислить среднее арифметическое для оценок.

## КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ПАМЯТИ (MEMORY-BASED)

Чтобы решить перечисленные выше проблемы, обратимся к коллаборативной фильтрации, а точнее к memory-based-подходу, основанному на близости пользователей (user-based).

Напомним, что при memory-based-подходе хранится полная матрица взаимодействий (лайков, просмотров и т. д .) пользователя с продуктом.

### КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ПОЛЬЗОВАТЕЛЕЙ (USER-BASED-ПОДХОД)

> **Коллаборативная фильтрация на основе пользователей** — это метод, используемый для предсказания продуктов, которые могут понравиться пользователю, на основе оценок, выставленных этому продукту другими пользователями, имеющими схожие с целевым пользователем вкусы.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5c33a47165a42a7a6dd473ac39459bfd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_2_ed.png)

В этом алгоритме мы заменяем жёсткую кластеризацию на следующую формулу и получаем предсказанную оценку пользователя $u$, которую он поставил элементу $i$:

$$\hat{r}_{u i}=\bar{r}_u+\frac{\sum_{v \in U_i} \operatorname{sim}(u, v)\left(r_{v i}-\bar{r}_v\right)}{\sum_{v \in U_i} \operatorname{sim}(u, v)}$$

Здесь используются следующие обозначения:

* $u$ и $v$ — индексы пользователей;
* $\bar{r}_u$ — средняя оценка пользователя ;
* $\bar{r}_v$ — средняя оценка пользователя ;
* $sim$ — функция схожести;
* $i$ — номер оцениваемого элемента.
Средняя оценка пользователя может быть никак не привязана к его интересам. По сути, это просто показатель того, как в среднем пользователь привык оценивать фильмы.

Оценка пользователя, которую мы предсказываем для него, состоит **из двух частей**:

* Непосредственно его средняя оценка.
* Слагаемое, состоящее из разницы в оценках с другими пользователями, т. е. похожести пользователей. Эта разница домножается на похожесть пользователей, то есть в числителе — средневзвешенная разница в оценках, а в знаменателе — сумма показателей схожести.

Каждому клиенту мы подбираем релевантный для него товар в рамках группы клиентов, но не решаем задачу кластеризации, а усредняем интересы данной группы в дистанции нескольких соседей. По сути, здесь мы руководствуемся идеей, что, например, видео можно порекомендовать человеку, если оно понравилось его друзьям.

Давайте разберём пример применения этого алгоритма.

В матрице четыре пользователя — Алиса, Рома, Катя и Женя. Они оценивают различные приложения из AppStore. Диапазон оценок — от 1 до 5. Знак '?' означает, что данный пользователь не оценил это приложение.

ИМЯ|APP1|APP2|APP3|APP4|APP5
-|-|-|-|-|-
Алиса|5|4|1|4|?
Рома|3|1|2|3|3
Катя|4|3|4|3|5
Женя|3|3|1|5|4

Вычислим сходство как коэффициент корреляции — такой подход также популярен.

Для начала найдём среднее значение рейтинга для каждого пользователя:

$$\bar{r}_i=\frac{\sum_p r_{i p}}{\sum p}$$

Таким образом получаем:

$\bar{r}_{\text {Алиса }}=3.5$

$\bar{r}_{Рома}=2.25$

$\bar{r}_{Катя}=3.5$

$\bar{r}_{Женя}=3$

Теперь вычисляем сходство между Алисой и всеми остальными пользователями:

$\operatorname{Sim}(Алиса, Рома)=\frac{((1.5 \cdot 0.75)+(0.5 \cdot (-1.25))+(-2.5 \cdot(-0.25))+(0.5 \cdot 0.75))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0.75^2+1.25^2+0.25^2+0.75^2\right)}}=0.301$

$\operatorname{Sim}(Алиса, Катя)=\frac{((1.5 \cdot 0.5)+(0.5 \cdot (-0.5))+(-2.5 \cdot 0.5)+(0.5 \cdot (-0.5))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0.5^2+0.5^2+0.5^2+0.5^2\right)}}=-0.33$

$\operatorname{Sim}(Алиса, Женя)=\frac{((1.5 \cdot 0)+(0.5 \cdot 0)+(-2.5 \cdot(-2))+(0.5 \cdot 2))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0^2+0^2+2^2+2^2\right)}}=0.707$

Теперь спрогнозируем рейтинг Алисы для приложения App5:

$r_{(\text {Алиса }, I 5)}=\bar{r}_{\text {Алиса }}+\frac{\left(\operatorname{sim}(\text { Алиса, } U 1) *\left(r_{U 1, I 5}-\bar{r}_{U 1}\right)\right)+\left(\operatorname{sim}(\text { Алиса,U2 }) *\left(r_{U 2, I 5}-\bar{r}_{U 2}\right)\right)+\left(\operatorname{sim}(Алиса, U 3) *\left(r_{U 3, I 5}-\bar{r}_{U 3}\right)\right.}{\operatorname{sim}(\text { Алиса, } U 1)+\operatorname{sim}(\text { Алиса,U2 })+\operatorname{sim}(Алиса, U 3)}$

$r_{(\text {Алиса, I5 })}=3.5+\frac{(0.301 * 0.75)+(-0.33 * 1.5)+(0.707 * 1)}{|0.301|+|-0.33|+|0.707|}=3.83$

Итак, мы смогли реализовать предсказание для user-based-подхода.



### КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ЭЛЕМЕНТОВ (ITEM-BASED-ПОДХОД)

Если мы транспонируем матрицу предпочтений и будем решать ту же самую задачу не для пользователей, а для объектов (items), то получим аналогичную задачу, которая является item-based-моделью коллаборативной фильтрации и даёт нам возможность предсказывать оценку следующим образом:

$$\hat{r}_{u i}=\bar{r}_i+\frac{\sum_{j \in I_u} \operatorname{sim}(i, j)\left(r_{u j}-\bar{r}_j\right)}{\sum_{j \in I_u} \operatorname{sim}(i, j)}$$

По формуле можно понять, что этот подход использует идею предыдущего, только теперь похожи не пользователи, а объекты. Возвращаясь к примеру с рекомендациями фильмов, теперь мы рекомендуем пользователю фильм, который похож на те фильмы, которые уже понравились этому пользователю ранее.

Кроме того, продуктов обычно больше, чем пользователей, поэтому векторы получатся большей размерности. Это даёт возможность получить более устойчивую модель с большей статистической значимостью.

Может показаться, что коллаборативная фильтрация в рамках item-based-подхода очень похожа на модель на основе контента. Однако это не так: item-based-модель рассматривает взаимодействия пользователей с продуктом, а content-based-модель — метаинформацию продукта.

Теперь давайте рассмотрим преимущества и недостатки коллаборативной фильтрации, основанной на памяти (memory-based):

**ПРЕИМУЩЕСТА**

* Может помочь пользователям обнаружить новые релевантные продукты из новых категорий, даже если пользователи не проявляют интерес к новым типам продуктов.
* Не требует подробных характеристик и контекстных данных о продуктах. По сути, для реализации нужна только матрица взаимодействий пользователей и продуктов.

**НЕДОСТАТКИ**

* Нехватка данных может привести к трудностям при рекомендации новых продуктов или при появлении новых пользователей, поскольку предложения основаны на исторических данных и взаимодействии. Таким образом, для этого подхода актуальна проблема холодного старта.
* По мере роста пользовательской базы алгоритмы страдают из-за большого объёма данных (очень ресурсоёмкие вычисления) и недостаточной масштабируемости.
* Отсутствие разнообразия в долгосрочной перспективе. Это может показаться нелогичным, поскольку смысл коллаборативной фильтрации заключается в том, чтобы рекомендовать пользователю новые товары. Однако поскольку алгоритмы функционируют на основе исторических рейтингов, они не будут рекомендовать товары с небольшим количеством оценок или ограниченным количеством данных. Популярные товары будут более популярны в долгосрочной перспективе, а новых и разнообразных вариантов будет не хватать.

Итак, мы рассмотрели memory-based-подходы коллаборативной фильтрации. Давайте перейдём к следующим подходам, которые относятся к категории model-based и основаны на разложениях матриц. После этого мы сможем сравнить изученные модели при решении практической задачи.

## MODEL-BASED-ПОДХОД

В подходе на основе моделей используются модели машинного обучения для прогнозирования и ранжирования взаимодействий между пользователями и элементами, с которыми они ещё не взаимодействовали. Эти модели обучаются на основе информации, уже имеющейся в матрице взаимодействий, с помощью различных алгоритмов, например матричной факторизации.

**Матричная факторизация** используется для генерации **латентных признаков** путём разложения **разрежённой** матрицы взаимодействия пользователя и продукта на две меньшие и **плотные** матрицы особенностей пользователей и продуктов.

- Под латентными признаками понимаются некоторые выделенные на основе данных факторы. Например, это может быть фактор приверженности определённому жанру фильмов.

- Мы получаем разрежённую матрицу из-за того, что продуктов очень много, и, очевидно, каждый пользователь взаимодействовал только с небольшим их количеством. Поэтому в матрице много пустых ячеек (нулей).

- Матрицы плотные, так как нам известны характеристики пользователей и продуктов.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d02212cd93a334020c696d2b7835e6eb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_3.png)

Слева на иллюстрации мы видим разрежённую матрицу (X), а справа — матрицу пользователей U с их характеристиками (размерности k) и матрицу товаров V с их характеристиками (размерности k).

Рассмотрим два варианта матричной факторизации — SVD и ALS.

SVD — это сингулярное разложение, с которым вы уже подробно знакомились в [модуле по кластеризации](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DST-3.0+28FEB2021/jump_to_id/de3e2256746440e1a97c14a54697424f). Давайте вспомним суть SVD.

Любую прямоугольную матрицу A размера (n, m) можно представить в виде произведения трёх матриц:

В этой формуле:

* $U$ — матрица размера $n,n$. Все её столбцы ортогональны друг другу и имеют единичную длину. Такие матрицы называются **ортогональными**. Эта матрица содержит нормированные собственные векторы матрицы $AA^T$.
* $D$ — матрица размера $n,m$. На её главной диагонали стоят числа, называемые **сингулярными** числами (они являются корнями из собственных значений матриц $AA^T$ и $A^TA$), а вне главной диагонали стоят нули. Если мы решаем задачу снижения размерности, то элементы этой матрицы, если их возвести в квадрат, можно интерпретировать как дисперсию, которую объясняет каждая компонента.
* $V$ — матрица размера $m,m$. Она тоже **ортогональная** и содержит нормированные собственные векторы матрицы $A^TA$).

Данное разложение используют для того, чтобы представить матрицу предпочтений как разложение на матрицу с характеристиками пользователей и матрицу с характеристиками продуктов. Матрица U представляет связь между пользователями и латентными факторами, D — диагональная матрица, описывающая силу каждого латентного фактора, а V — матрица, описывающая связь между продуктами и латентными факторами.

Однако SVD — не единственная возможность для разложения матрицы. Второй популярный алгоритм — ALS (Alternating Least Square).

ALS — итеративный алгоритм разложения матрицы предпочтений на произведение двух матриц.

Чтобы понять лучше суть этого алгоритма, вспомним одну из известных нам функций потерь — RMSE.

Предположим, что есть m пользователей и n продуктов. Тогда у нас будут следующие матрицы:

* $R$ размерности $m*n$;
* $U$ размерности $m*k$;
* $P$ размерности $n*k$, где $k$ — количество латентных факторов.

Тогда функция потерь, которую мы будем минимизировать, будет следующей:

$$\begin{aligned} \text { loss } & =\min (y-\hat{y})^2 \\ & =\min \left(R-U * P^T\right)^2 \\ & =\min \sum_{m, n}\left(R_{m, n}-U_m * P_n^T\right)^2\end{aligned}$$

Здесь:

* $R$ — истинные показатели взаимодействия пользователя и продукта;
* $U*P^T$ — прогнозируемые показатели взаимодействия пользователя и продукта.

Далее мы, по сути, решаем метод наименьших квадратов для разницы матриц, на каждом шаге уменьшая ошибку сначала по факторам пользователей, а затем — по факторам товаров. Чтобы избежать ситуации переобучения, к ошибке добавляются регуляризационные коэффициенты.

ALS — это итерационный процесс оптимизации, в котором мы на каждой итерации пытаемся приблизиться к факторизованному представлению исходных данных.

Давайте рассмотрим реализацию ALS на «игрушечном» примере. Допустим, у нас есть некоторая матрица, которая содержит информацию про пользователей и их отношение к фильмам:

/|ФИЛЬМ 1|ФИЛЬМ 2|ФИЛЬМ 3
-|-|-|-
Пользователь 1|0.5|?|4
Пользователь 2|1|3|5

Однако нам неизвестно, какую оценку поставил Пользователь 1 Фильму 2 — её мы и будем пытаться предсказать с помощью ALS.

В соответствии с методом ALS мы хотим получить следующее разложение:

$\left(\begin{array}{ccc}0.5 & ? & 4 \\ 1 & 3 & 5\end{array}\right)=\left(\begin{array}{l}u_1 \\ u_2\end{array}\right)\left(\begin{array}{lll}p_1 & p_2 & p_3\end{array}\right)$

Для начала фиксируем матрицу U (вектор-столбец). Для неё можно выбрать любые случайные числа, но для удобства вычислений примем оба значения за единицу:

$\left(\begin{array}{ccc}0.5 & ? & 4 \\ 1 & 3 & 5\end{array}\right)=\left(\begin{array}{l}1 \\ 1\end{array}\right)\left(\begin{array}{lll}p_1 & p_2 & p_3\end{array}\right)$

Если мы перемножим матрицы в правой части равенства, то получим пять уравнений, в которых участвуют компоненты из вектора p:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7cfd0f260ead1de4a75aa302d7108876/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq4.png)

Поскольку существует единственное уравнение, определяющее вторую компоненту P, мы задаём её равной 3. Нам необходимо выбрать оставшиеся две компоненты так, чтобы средняя квадратичная ошибка была минимальной. Таким образом, вычисляем:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/279fe917219154ac3ca65f2fb52443c0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq5.png)

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/8ca933222d3532e8cc93cc2f8b5d79ae/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq6.png)

Разумеется, мы умеем вычислять точки минимума для функции, используя производные. Находим точки минимума для обоих случаев и получаем первую оценку для матрицы:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7ed8812fc95a1f755772a90238182c97/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq7.png)

Теперь мы оставляем компоненты матрицы P фиксированными и оптимизируем матрицу U. Это аналогично даёт нам следующие уравнения для U:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/099c4c6f2311990b9a03599c4e81f1f6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq8.png)

Далее мы можем минимизировать среднеквадратичную ошибку для компонентов матрицы U, как мы это делали ранее для компонентов матрицы P.

Повторяя эти итерации, мы в какой-то момент сходимся к оптимальным матрицам U и P. В данном примере после 20 итераций можно определить, что U и P даны в виде:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/f6ef822ea53016f0a6d20afb87d1d906/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq9.png)

Теперь из этих матриц мы можем получить восстановленную матрицу прогнозируемых взаимодействий пользователей и продуктов, на основе которой мы можем составлять рекомендации.

In [12]:
from surprise import Dataset

ModuleNotFoundError: No module named 'surprise'